In [1]:
from ultralytics import YOLO
import torch
import gc
import os

# Solution pour la fragmentation mémoire
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Nettoyage mémoire agressif
torch.cuda.empty_cache()
gc.collect()

# Load model
model = YOLO("yolo11n.pt")

DATA_YAML = "VisDrone.yaml"
RUN_NAME = "visdrone_detection_fast" 

train_kwargs = dict(
    data=DATA_YAML,
    epochs=50,
    imgsz=640,
    batch=16,           # RÉDUIT à 8 (au lieu de 16)
    cache="disk",       # "ram" si assez de RAM, sinon "disk" ou False
    device=0,
    workers=2,         # RÉDUIT workers pour moins de mémoire
    resume=False,
    save_period=5,     # Réduit la fréquence des sauvegardes
    half=True,
    compile=False,     # DÉSACTIVE compile (peut causer OOM)
    patience=10,       # Arrêt précoce si pas d'amélioration
    lr0=0.005,          # Learning rate plus bas
    warmup_epochs=2,   # Warmup pour stabiliser
    close_mosaic=10,   # Désactive mosaic à la fin
)
print(f"Training with batch size: {train_kwargs}")

Training with batch size: {'data': 'VisDrone.yaml', 'epochs': 50, 'imgsz': 640, 'batch': 16, 'cache': 'disk', 'device': 0, 'workers': 2, 'resume': False, 'save_period': 5, 'half': True, 'compile': False, 'patience': 10, 'lr0': 0.005, 'warmup_epochs': 2, 'close_mosaic': 10}


In [2]:
# Train the model
results = model.train(**train_kwargs)

New https://pypi.org/project/ultralytics/8.3.230 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.221 🚀 Python-3.12.3 torch-2.9.0+cu128 CUDA:0 (NVIDIA RTX A1000 6GB Laptop GPU, 5807MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=disk, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=VisDrone.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=True, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train8, nbs=64, nms=False, opset=N

In [2]:
MODEL_PRETRAINED = YOLO("/media/Data/uav/uav-target-detection/uav_1/runs/detect/train/weights/best.pt")

In [ ]:
# Évaluation sur le dataset de validation
results = model.val(data='VisDrone.yaml', split='val') 

Ultralytics 8.3.221 🚀 Python-3.12.3 torch-2.9.0+cu128 CUDA:0 (NVIDIA RTX A1000 6GB Laptop GPU, 5807MiB)
val: Fast image access ✅ (ping: 0.1±0.0 ms, read: 182.2±60.8 MB/s, size: 110.2 KB)
val: Scanning /media/Data/uav/uav-target-detection/uav_1/dataset/visdrone/labels/val.cache... 548 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 548/548 332.3Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 35/35 3.7it/s 9.4s0.1s
                   all        548      38759     0.0354      0.136     0.0312     0.0144
                person        520       8844     0.0909      0.281      0.145     0.0625
               bicycle        482       5125      0.101      0.066     0.0554     0.0183
                   car        364       1287    0.00247      0.109    0.00139   0.000404
            motorcycle        515      14064      0.052     0.0235     0.0266     0.0175
              airplane        421       1975     0.0187    0.00

In [ ]:
Métriques Globales
mAP50: 0.312 (31.2%) → 10x mieux que précédemment !

mAP50-95: 0.181 (18.1%) → Amélioration significative

Precision: 0.419 (41.9%) → Beaucoup moins de faux positifs

Recall: 0.315 (31.5%) → Plus d'objets détectés